# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [16]:
import pandas as pd
import numpy as np
import folium
from scipy.spatial import distance

In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: False


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [4]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [5]:
from sklearn.cluster import KMeans

In [6]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [7]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [8]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [9]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [10]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [11]:
pred_centers = myKMeans.predict(coordinates)

coordiantes2 = coordinates.copy() #deep copy
coordiantes2.insert(4, "center", pred_centers, True) 
coordiantes2

,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,center
0,40.721319,-73.844311,40.712278,-73.841610,59
1,40.711303,-74.016048,40.782004,-73.979268,31
2,40.761270,-73.982738,40.750562,-73.991242,43
3,40.733143,-73.987130,40.758092,-73.991567,81
4,40.768008,-73.968095,40.783762,-73.956655,56
...,...,...,...,...,...
399995,40.746032,-73.986585,40.724077,-73.990865,51
399996,40.742359,-73.992882,40.762318,-73.972649,46
399997,40.731558,-73.985598,40.728738,-73.987657,18
399998,40.740735,-74.007692,40.722847,-73.988455,2


In [12]:
def show_cluster(cluster_number, pMap):
    start_end = coordiantes2[coordiantes2["center"] == cluster_number]
    #print(start_end)
    for i in range(len(start_end)):
        row = start_end.iloc[i]
        folium.CircleMarker([row[0], row[1]], radius=3,                
                       color="green", 
                       fill_opacity=0.4
                       ).add_to(pMap)
        folium.CircleMarker([row[2], row[3]], radius=3,                
                       color="red", 
                       fill_opacity=0.4
                       ).add_to(pMap)

In [13]:
def create_map(cluster_center):
    cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
    i = cluster_center
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=20,                
                        color="green",
                        fill=True,
                        fill_opacity=0.3
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=20,                
                        color="red",
                        fill=True,
                        fill_opacity=0.3
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)
    return cluster_map

In [14]:
cluster_num = 99
c_map = create_map(cluster_num)
show_cluster(cluster_num, c_map)
c_map

## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.

In [17]:
def intra_cluster_var(rows, row_index):    
    i = rows.iloc[row_index]
 
    # if i = j distance is 0 -> i != j if we sum up        
    ii = [i for j in range(len(rows))]
        
    res = distance.cdist(ii, rows)
    return (1/len(rows)) * np.sum(res)

def cluster_var(cluster_number):
    start_end_rows = coordiantes2[coordiantes2["center"] == cluster_number]
    del start_end_rows["center"]
 
    res = [intra_cluster_var(start_end_rows, i) for i in range(len(start_end_rows))] 
    print(res)
    

cluster_var(30)

[0.35350971987343704, 0.35350971987343704]
